**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

Решение первой задачи с использованием Pandas

In [2]:
%%time

def user_lifetime(user_id):
    """вычисляет время жизни пользователя"""
    return (
      ratings.loc[(ratings.userId == user_id)].max()['timestamp'] - ratings.loc[(ratings.userId == user_id)].min()['timestamp']
    )

ratings = pd.read_csv('ratings.csv')
users = ratings.groupby('userId').count()
more_100 = users.loc[(users.rating > 100)].reset_index()

more_100['lifetime'] = more_100['userId'].apply(user_lifetime)
averege_lifetime = more_100['lifetime'].mean() // 2629743
print(averege_lifetime, 'месяцев')

15.0 месяцев
Wall time: 698 ms


Решение первой задачи без Pandas

In [3]:
%%time

def user_lifetime(user_data):
    """вычисляет время жизни пользователя"""
    return max([x[1] for x in user_data]) - min([x[1] for x in user_data])

previous_user = None
user_data = []
all_users_lifetimes = []
with open('ratings.csv') as f:
    header = f.readline()
    for line in f:
        line = line.strip().split(',')
        user_id, movieId, rating, timestamp = line
        timestamp = int(timestamp)
        
        if previous_user == None:
            previous_user = user_id
        else:
            if user_id == previous_user:
                user_data.append([user_id, timestamp])
            else:
                if len(user_data) > 100:
                    life_time = user_lifetime(user_data)
                    all_users_lifetimes.append(life_time)
             
                user_data = []
                user_data.append([user_id, timestamp])
                previous_user = user_id

life_time = user_lifetime(user_data)
all_users_lifetimes.append(life_time)

average_lifetime = (sum(all_users_lifetimes) / len(all_users_lifetimes))  // 2629743
print(average_lifetime, 'месяцев')    


15.0 месяцев
Wall time: 306 ms


**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки,
auto - автомобильные перевозки,
air - воздушные перевозки,
client_base - адреса клиентов


In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [5]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [6]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [7]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [8]:
rzd.merge(auto,  how='outer').merge(air,  how='outer').fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [9]:
client_base.merge(rzd, how='left').merge(auto,  how='left').merge(air,  how='left').fillna(0)

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,0.0,0.0
1,112,Энтузиастов 8а,2810.0,0.0,0.0
2,113,Левобережная 1а,10283.0,57483.0,0.0
3,114,Мира 14,5774.0,83.0,0.0
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,0.0,4834.0,4.0
6,117,Панфиловская 33,0.0,98.0,13.0
7,118,Мастеркова 4,0.0,0.0,173.0


**Задание 3**

 В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Можно создать в таблицах визитов и покупок две новые колонки: latitude / longitude и занести в них данные широты/долготы.

По координатам можно узнать страну, регион нахождения пользователя, город, район города.

А так же насколько часто пользователь путешествует.

Если раз-два в год -> скорее всего это отпуск.

Если раз-два в месяц -> возможно его работа связана с командировками.

Эти данные вполне можно использовать для рекламы товаров и услуг.